In [6]:
import os
import datetime
from pathlib import Path

def collect_project_info(root_path, output_file="project_complete_info.txt"):
    """
    Thu thập cấu trúc thư mục và nội dung tất cả file code
    """
    # Các extension file code phổ biến
    code_extensions = {
        '.py', '.js', '.css', '.json', '.xml', '.yml', '.yaml',
        '.md', '.txt', '.sql', '.sh', '.bat', '.cfg', '.ini', '.env',
        '.jsx', '.ts', '.tsx', '.vue', '.php', '.java', '.cpp', '.c',
        '.h', '.cs', '.rb', '.go', '.rs', '.swift', '.kt'
    }
    
    # Thư mục và file cần bỏ qua
    ignore_dirs = {
        '__pycache__', '.git', '.vscode', '.idea', 'node_modules', 
        '.pytest_cache', '.mypy_cache', 'venv', 'env', '.env',
        'dist', 'build', '.next', '.nuxt'
    }
    
    ignore_files = {
        '.pyc', '.pyo', '.pyd', '.so', '.dll', '.exe', '.log',
        '.tmp', '.temp', '.cache', '.DS_Store', 'Thumbs.db'
    }
    
    # THÊM: Danh sách file cụ thể cần bỏ qua
    ignore_specific_files = {
        'employees.json', 'depots.json', 'transport_providers.json'
    }
    
    root_path = Path(root_path)
    
    # SỬA LỖI: Đảm bảo output_file là string
    if isinstance(output_file, Path):
        output_file = str(output_file)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        # Header thông tin
        f.write("=" * 80 + "\n")
        f.write("THÔNG TIN DỰ ÁN HOÀN CHỈNH\n")
        f.write("=" * 80 + "\n")
        f.write(f"Thời gian tạo: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Thư mục gốc: {root_path.absolute()}\n")
        f.write("=" * 80 + "\n\n")
        
        # 1. Cấu trúc thư mục
        f.write("📁 CẤU TRÚC THU MỤC\n")
        f.write("-" * 50 + "\n")
        write_directory_tree(f, root_path, ignore_dirs, ignore_specific_files)
        f.write("\n\n")
        
        # 2. Danh sách tất cả file
        f.write("📄 DANH SÁCH TẤT CẢ FILE\n")
        f.write("-" * 50 + "\n")
        all_files = list_all_files(root_path, ignore_dirs, ignore_files, ignore_specific_files)
        for file_path in sorted(all_files):
            rel_path = file_path.relative_to(root_path)
            file_size = file_path.stat().st_size
            f.write(f"{rel_path} ({file_size} bytes)\n")
        f.write(f"\nTổng cộng: {len(all_files)} file\n\n")
        
        # 3. Nội dung tất cả file code
        f.write("💻 NỘI DUNG TẤT CẢ FILE CODE\n")
        f.write("=" * 80 + "\n")
        
        code_files = [f for f in all_files if f.suffix.lower() in code_extensions]
        
        for file_path in sorted(code_files):
            rel_path = file_path.relative_to(root_path)
            f.write(f"\n{'='*60}\n")
            f.write(f"FILE: {rel_path}\n")
            f.write(f"{'='*60}\n")
            
            try:
                # Thử đọc với UTF-8 trước
                content = file_path.read_text(encoding='utf-8')
                f.write(content)
            except UnicodeDecodeError:
                try:
                    # Nếu không được thì thử với encoding khác
                    content = file_path.read_text(encoding='latin-1')
                    f.write(content)
                except Exception as e:
                    f.write(f"[KHÔNG THỂ ĐỌC FILE: {e}]\n")
            except Exception as e:
                f.write(f"[LỖI ĐỌC FILE: {e}]\n")
            
            f.write(f"\n{'='*60}\n")
        
        # 4. Thống kê cuối
        f.write(f"\n\n📊 THỐNG KÊ\n")
        f.write("-" * 30 + "\n")
        f.write(f"Tổng số file: {len(all_files)}\n")
        f.write(f"File code: {len(code_files)}\n")
        f.write(f"Các loại file code:\n")
        
        extensions = {}
        for file in code_files:
            ext = file.suffix.lower()
            extensions[ext] = extensions.get(ext, 0) + 1
        
        for ext, count in sorted(extensions.items()):
            f.write(f"  {ext}: {count} file\n")

def write_directory_tree(f, path, ignore_dirs, ignore_specific_files, prefix=""):
    """Viết cấu trúc thư mục dạng tree"""
    try:
        items = sorted(path.iterdir(), key=lambda x: (x.is_file(), x.name.lower()))
        
        for i, item in enumerate(items):
            if item.name.startswith('.') and item.name not in {'.env', '.gitignore'}:
                continue
            if item.name in ignore_dirs:
                continue
            # THÊM: Bỏ qua file cụ thể
            if item.name in ignore_specific_files:
                continue
                
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            
            if item.is_dir():
                f.write(f"{prefix}{current_prefix}📁 {item.name}/\n")
                next_prefix = prefix + ("    " if is_last else "│   ")
                write_directory_tree(f, item, ignore_dirs, ignore_specific_files, next_prefix)
            else:
                f.write(f"{prefix}{current_prefix}📄 {item.name}\n")
                
    except PermissionError:
        f.write(f"{prefix}[KHÔNG CÓ QUYỀN TRUY CẬP]\n")

def list_all_files(root_path, ignore_dirs, ignore_files, ignore_specific_files):
    """Lấy danh sách tất cả file"""
    all_files = []
    
    for item in root_path.rglob("*"):
        # Bỏ qua thư mục ignore
        if any(ignore_dir in item.parts for ignore_dir in ignore_dirs):
            continue
            
        # Bỏ qua file ẩn (trừ một số file quan trọng)
        if item.name.startswith('.') and item.name not in {'.env', '.gitignore', '.htaccess'}:
            continue
            
        # Bỏ qua file theo extension
        if item.suffix.lower() in ignore_files:
            continue
        
        # THÊM: Bỏ qua file cụ thể theo tên
        if item.name in ignore_specific_files:
            continue
            
        if item.is_file():
            all_files.append(item)
    
    return all_files

# Sử dụng script
if __name__ == "__main__":
    # Thay đổi đường dẫn này thành thư mục dự án của bạn
    project_path = "."  # Thư mục hiện tại
    output_file = "project_complete_info.txt"  # Đảm bảo là string
    
    print("🔄 Đang thu thập thông tin dự án...")
    collect_project_info(project_path, output_file)
    print(f"✅ Hoàn thành! Thông tin đã được lưu vào: {output_file}")
    print(f"📁 File size: {os.path.getsize(output_file)} bytes")

🔄 Đang thu thập thông tin dự án...
✅ Hoàn thành! Thông tin đã được lưu vào: project_complete_info.txt
📁 File size: 256052 bytes


In [7]:
# import requests
# import json
# from datetime import datetime

# # Cấu hình App credentials
# APP_ID = "cli_a7fab27260385010"
# APP_SECRET = "Zg4MVcFfiOu0g09voTcpfd4WGDpA0Ly5"

# # Cấu hình LarkBase
# APP_TOKEN = "Ey3EbVD9vacAHvs8cVvlHxkKg2r"
# TABLE_ID = "tblHDR4RsHUCS7Aa"

# def get_tenant_access_token(app_id, app_secret):
#     """
#     Lấy tenant access token từ App ID và App Secret
#     """
#     url = "https://open.larksuite.com/open-apis/auth/v3/app_access_token/internal"
    
#     headers = {
#         "Content-Type": "application/json"
#     }
    
#     payload = {
#         "app_id": app_id,
#         "app_secret": app_secret
#     }
    
#     response = requests.post(url, headers=headers, json=payload)
#     result = response.json()
    
#     if result.get("code") == 0:
#         print("✓ Lấy token thành công!")
#         return result.get("app_access_token")
#     else:
#         print(f"✗ Lỗi khi lấy token: {result}")
#         return None

# def filter_records_by_date(access_token, app_token, table_id, date_timestamp):
#     """
#     Lọc records theo ngày sử dụng filter
#     """
#     url = f"https://open.larksuite.com/open-apis/bitable/v1/apps/{app_token}/tables/{table_id}/records/search"
    
#     headers = {
#         "Authorization": f"Bearer {access_token}",
#         "Content-Type": "application/json"
#     }
    
#     # Filter theo ngày với operator "is"
#     filter_condition = {
#         "conditions": [
#             {
#                 "field_name": "Date",  # Tên field trong LarkBase
#                 "operator": "is",
#                 "value": ["ExactDate", str(date_timestamp)]
#             }
#         ],
#         "conjunction": "and"
#     }
    
#     payload = {
#         "filter": filter_condition,
#         "automatic_fields": False  # Có lấy các field tự động hay không
#     }
    
#     response = requests.post(url, headers=headers, json=payload)
#     result = response.json()
    
#     return result

# def main():
#     print("=== TEST FILTER NGÀY TRONG LARKBASE ===\n")
    
#     # Bước 1: Lấy access token
#     print("1. Đang lấy access token...")
#     access_token = get_tenant_access_token(APP_ID, APP_SECRET)
    
#     if not access_token:
#         print("Không thể tiếp tục do lỗi xác thực!")
#         return
    
#     print(f"Access Token: {access_token[:20]}...\n")
    
#     # Bước 2: Thực hiện filter
#     print("2. Đang filter records theo ngày...")
    
#     # Timestamp từ yêu cầu của bạn
#     date_timestamp = "1761693696000"
    
#     # Chuyển timestamp sang định dạng readable để kiểm tra
#     date_readable = datetime.fromtimestamp(int(date_timestamp) / 1000)
#     print(f"Ngày cần filter: {date_readable.strftime('%Y-%m-%d %H:%M:%S')}\n")
    
#     # Gọi API filter
#     result = filter_records_by_date(access_token, APP_TOKEN, TABLE_ID, date_timestamp)
    
#     # Bước 3: Hiển thị kết quả
#     print("3. Kết quả:")
#     print(json.dumps(result, indent=2, ensure_ascii=False))
    
#     if result.get("code") == 0:
#         items = result.get("data", {}).get("items", [])
#         print(f"\n✓ Tìm thấy {len(items)} record(s) khớp với điều kiện filter")
        
#         # Hiển thị chi tiết các records
#         for idx, item in enumerate(items, 1):
#             print(f"\n--- Record {idx} ---")
#             print(f"Record ID: {item.get('record_id')}")
#             print(f"Fields: {json.dumps(item.get('fields'), indent=2, ensure_ascii=False)}")
#     else:
#         print(f"\n✗ Lỗi: {result.get('msg')}")

# if __name__ == "__main__":
#     main()


In [8]:
# import lark_oapi as lark
# from lark_oapi.api.bitable.v1 import *
# import json
# from datetime import datetime


In [9]:
# # App credentials
# APP_ID = "cli_a7fab27260385010"
# APP_SECRET = "Zg4MVcFfiOu0g09voTcpfd4WGDpA0Ly5"

# # LarkBase configuration
# APP_TOKEN = "Rm9PbvKLeaFFZcsSQpElnRjIgXg"
# TABLE_ID = "tblJJPUEFhsXHaxY"

# # Timestamps
# start_date = datetime(2025, 10, 20, 0, 0, 0)
# START_TIMESTAMP = str(int(start_date.timestamp() * 1000))

# end_date = datetime(2025, 10, 30, 23, 59, 59)
# END_TIMESTAMP = str(int(end_date.timestamp() * 1000))

# # Tạo client
# client = lark.Client.builder() \
#     .app_id(APP_ID) \
#     .app_secret(APP_SECRET) \
#     .log_level(lark.LogLevel.INFO) \
#     .build()

# print("✓ Client initialized")
# print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
# print(f"START_TIMESTAMP: {START_TIMESTAMP}")
# print(f"END_TIMESTAMP: {END_TIMESTAMP}")


In [10]:
# print("="*80)
# print("FETCH ALL RECORDS - AUTOMATIC PAGINATION")
# print("="*80)

# all_records = []
# all_record_ids = []
# page_token = None
# page_count = 0
# max_pages = 20  # Safety limit

# while page_count < max_pages:
#     page_count += 1
#     print(f"\n--- Page {page_count} ---")
    
#     # Tạo request builder
#     request_builder = SearchAppTableRecordRequest.builder() \
#         .app_token(APP_TOKEN) \
#         .table_id(TABLE_ID) \
#         .page_size(500)
    
#     # Thêm page_token nếu có
#     if page_token:
#         request_builder.page_token(page_token)
#         print(f"Using page_token: {page_token[:50]}...")
#     else:
#         print("First request (no page_token)")
    
#     # Build request với filter
#     request = request_builder.request_body(
#         SearchAppTableRecordRequestBody.builder()
#             .filter(FilterInfo.builder()
#                 .conjunction("and")
#                 .conditions([
#                     Condition.builder()
#                         .field_name("Ngày bàn giao")
#                         .operator("isGreater")
#                         .value(["ExactDate", START_TIMESTAMP])
#                         .build(),
#                     Condition.builder()
#                         .field_name("Ngày bàn giao")
#                         .operator("isLess")
#                         .value(["ExactDate", END_TIMESTAMP])
#                         .build()
#                 ])
#                 .build())
#             .automatic_fields(False)
#             .build()
#     ).build()
    
#     # Gửi request
#     response = client.bitable.v1.app_table_record.search(request)
    
#     if not response.success():
#         print(f"❌ Error: {response.code} - {response.msg}")
#         break
    
#     # Xử lý response
#     data = response.data
#     items = data.items or []
#     has_more = data.has_more
#     page_token = data.page_token
    
#     # Lưu records
#     current_ids = [item.record_id for item in items]
#     all_records.extend(items)
#     all_record_ids.extend(current_ids)
    
#     print(f"Retrieved: {len(items)} records")
#     print(f"Total so far: {len(all_records)}")
#     print(f"has_more: {has_more}")
    
#     # Dừng nếu không còn data
#     if not has_more or len(items) == 0:
#         print(f"\n✓ Completed! No more data.")
#         break

# print("\n" + "="*80)
# print("FINAL SUMMARY")
# print("="*80)
# print(f"Total pages: {page_count}")
# print(f"Total records: {len(all_records)}")
# print(f"Unique records: {len(set(all_record_ids))}")
# print(f"Duplicate records: {len(all_record_ids) - len(set(all_record_ids))}")
